# Multivariate Regression Hypothesis testing with Vector Spaces

```
Author:
Zach Wolpe
zachcolinwolpe@gmail.com
zach.world
```

In [250]:
import numpy as np
import pandas as pd
import scipy.stats as st

# Basic Regression Calculations

Clean data and compute the standard regression values of interest

In [130]:
# define dependent/independent variables and add intercept
X = data.data
X = sm.add_constant(X)
y = data.target

In [131]:
# RAL dataset
df = pd.read_csv('RAL_data.txt')
y = df.iloc[:,0]
X = df.iloc[:, 1:]
X = sm.add_constant(X)

In [114]:
# compute the Hat matrix / Projection matrix
inv = np.linalg.inv
XPX = np.dot(X.T,X)
invXPX = inv(XPX)
H = np.dot(np.dot(X, invXPX),X.T)

# sums
n = X.shape[0] 
p = X.shape[1]
I = np.identity(n)
sumY = np.sum(np.array(y))
sumX = np.sum(X)
XY = np.dot(X.T, y)
XX = np.dot(X.T, X)
yy = np.dot(y.T, y)

########################################## Estimates ##########################################

# betas
b = np.dot(inv(XX), XY)

# Y_hat
yhat = np.dot(H,y)
yhat = np.dot(X,b)

# error
e = y - yhat
e = np.dot((I-H),y)

####################################### Sum of Squares #######################################

# sse
sse = np.dot(e.T, e)

# sstou
sstou = np.dot(y.T, y)

# ss1 
ss1 = 1/n*(sumY**2)

# ssx
ssx = np.dot(y.T, yhat)
ssx = np.dot(np.dot(b.T, X.T),y)

# ssr 
ssr = ssx - ss1

# ssto 
ssto = sstou - ss1

####################################### Mean Squares #######################################

# MSE 
mse = sse/(n-p)

# MSR
msr = ssr/(p-1)

########################################### ANOVA ###########################################


# F statistic
F = msr/mse

# F critical value
F_crit = st.f.ppf(q=(1-0.05), dfn=(p-1), dfd=(n-p))

# p-value
p_value = 1 - st.f.cdf(F, p-1, n-p)


######################################## Correlation ########################################



# R squared (coefficient of multiple determination)
R2 = ssr/ssto

# adjusted R squared
R2_adjusted = 1 - (sse/(n-p))/(ssto/(n-1))

# coefficient of multiple correlation
r = np.sqrt(R2)


######################################### Variances #########################################

# var{beta} (covariance matrix of Beta's)
covb = mse*invXPX

# var{Yhat}
varYh = mse*H

# var{e}
varE = mse*(I-H)

# Calculate Beta Confidence intervals

Allows us to test any parameter estimates confidence interval. 

In [115]:
def betas_CI(X, y, alpha=0.05):
    """
    Allows us to test any parameter estimates confidence interval. 
    T distribution Confidence Intervals
    
    X:         matrix of X independent variables
    y:         target (response) variable
    alpha:     level of significance
    center:    number to center CI around / add later
    """
    
    n = y.shape[0]
    p = X.shape[1]
    XPX = np.dot(X.T,X)
    sstou = np.dot(y.T, y)
    np.linalg.inv(XPX)
    b = np.dot(invXPX, np.dot(X.T, y))
    ssx = np.dot(b.T, np.dot(X.T, y))
    sse = sstou - ssx
    mse = sse/(n-p)
    
    covb = mse*invXPX
    
    
    intervals = []
    for i in range(X.shape[1]):
        
        std = np.sqrt(covb[i,i])
        

        # interval
        add = ((b[i] - st.t.ppf(1-(alpha/2),df=n-p)*std), (b[i] + st.t.ppf(1-(alpha/2),df=n-p)*std))
        intervals.append(add)
        
    return np.array(intervals)

In [168]:
CIs = betas_CI(X, y, alpha=0.05)
print('')
print('confidence intervals for betas (alpha=0.05): ')
print('')
print(CIs)


confidence intervals for betas (alpha=0.05): 

[[121.91172735 195.07077598]
 [ -1.57509339  -0.7081303 ]
 [ -1.43483133   0.55082281]
 [-27.79785874   0.85753235]]


# Partial F test

Any regression hypothesis can be specified in terms of Full and Reduced models, wherein the reduced model assumes $H_0$ to be true. 

Most commonly used to test Extra sum of squares in the General Linear Approach.

## Extra Sum of Squares

$$SSR(X_i|X_j,X_k) = SSR(X_i,X_j,X_k) - SSR(X_j,X_k)$$

To test the partial effect of adding an independent variable, given the current model specification, we can assess the extra sum of squares - which simply calculates the increase in regression SS or decrease in error SS if additional explanatory variables are added to the model.

### Coefficient of Partial Determination
$$R^2_{1|2} = \frac{SSR(X1 | x2)}{SSE(X2)}$$

Measures the proportion reduction of  $SSE(X2)$ (in this example) when $X1$ is added to the model. Calculates the marginal reduction in variance of $Y$ when $X1$ is added to a model containing $X2$


### Degrees of Freedom
$df_i$: An Extra SS has degrees of freedom $=$ no. additional $X$ variables

## Partial F-test
F-test can be used to test if the marginal effects are measurable. The F test is given in terms of the _Full Model_ & _Reduced Model_:

$$F^* = \frac{SSR(X_i|X_j, X_k)/df_i}{SSE(X_j, X_k, X_i)/(n-p)} \sim F(df_i, n-p)$$

Specify research questions such that:

$$H_0: Reduced Model$$
$$H_a: Full Model$$



In [247]:
def partial_F_test(y, reduced, full, alpha=0.05):
    """
    Derive the F statistics for an extra sum of squares

    y:     vector, target variable (n x 1)
    
    full: all X var in the full model
    reduced: all X vars in the reduced model
    """
    
    # convert to dataframe
    reduced = pd.DataFrame(reduced)
    full = pd.DataFrame(full)
    y = pd.DataFrame(y)
    
    sstou = np.dot(y.T, y)
    
    # reduced model calculations
    if (reduced.shape[1] > 1):
        inv_xpx_r = np.linalg.inv(np.dot(reduced.T, reduced))
    else:
        inv_xpx_r = np.reciprocal(np.dot(reduced.T, reduced))
        
    XY_r = np.dot(reduced.T, y)
    ssx_r = np.dot(np.dot(np.dot(y.T, reduced), inv_xpx_r), XY_r)
    sse_r = sstou - ssx_r
    df_r = reduced.shape[0] - reduced.shape[1]
    
    
    
    # full model calculations
    if (reduced.shape[1] > 1):
        inv_xpx_f = np.linalg.inv(np.dot(full.T, full))
    else:
        inv_xpx_f = np.reciprocal(np.dot(full.T, full))
        
        
    XY_f = np.dot(full.T, y)
    ssx_f = np.dot(np.dot(np.dot(y.T, full), inv_xpx_f), XY_f)
    sse_f = sstou - ssx_f
    df_f = full.shape[0] - full.shape[1]
    
    mseF = sse_f/df_f
    
    # extra SS
    extra_SS = ssx_f - ssx_r
    
    # F statistic
    F = ((sse_r-sse_f)/(df_r - df_f))/mseF
    
    
    # critical value
    F_crit = st.f.ppf(1-alpha, df_r, df_f)
    
    # p value
    p_value = st.f.pdf(F, df_r, df_f)
    
    # Partial R Squared
    partial_R2 = extra_SS/sse_r
    
    
    results = {
        'F statistic': F,
        'extra SS': extra_SS,
        'critcal value': F_crit,
        'p value': p_value,
        'partial R2': partial_R2,
    }
    
    return results

In [249]:
reduced_model = pd.concat([X.iloc[:,0], X.iloc[:,2]], axis=1)
full_model = X.iloc[:,0:3]
res = partial_F_test(y, reduced_model, full_model, alpha=0.05)
res

{'F statistic': array([[36.31690673]]),
 'extra SS': array([[3896.04414241]]),
 'critcal value': 1.6572460294627507,
 'p value': array([[2.46261102e-23]]),
 'partial R2': array([[0.45787094]])}

# Hypothysis testing: Matrix notation

Analogous to the above F test, however specified in Matrix notation.

Test any Hypothesis specified in the form:
$$H_0: LB = c$$
$$H_0: LB \neq c$$

By computing the $F^*$ statistic:

$$F^* = \frac{(LB - c)` (L(X`X)^{-1}L`)^{-1}(LB - c) /r}{MSE} \sim F(1-a, r, n-p)$$


let $Q = (LB - c)` (L(X`X)^{-1}L`)^{-1}(LB - c)$


## Degrees of freedom
#### numerator
$r$: no. linear independent functions in $L$

#### denominator
$n-p$: sse df from the full model

In [236]:
def LB_F_statistics(L, X, y, c, alpha=0.05):
    """
    Calculate the definitive statistics for an
    H0: LB = c
    hypothesis test.
    
    Parameters:
    L & c: (vector/matrix) hypothesis parameters
    X: (vector/matrix) independent variables
    y: (vector) target variable
    alpha: level of significance
    """
    # calculate Beta's
    XX = np.dot(X.T, X)
    XY = np.dot(X.T, y)
    invXX = np.linalg.inv(XX)
    b = pd.DataFrame(np.dot(invXX, XY))
    
    # calculate Q
    r = L.shape[0]
    Lb = np.dot(L, b)
    if np.array([np.dot(np.dot(L,invXX), L.T)]).shape[0] > 2:
        Q_center = np.linalg.inv(np.dot(L, np.dot(invXX, L.T)))
    else:
        Q_center = np.reciprocal(np.dot(L, np.dot(invXX, L.T)))
    Q = np.dot(np.dot((Lb-c).T, Q_center), (Lb-c))
    
    
    # calculate MSE
    n = X.shape[0]
    p = X.shape[1]
    sse = np.dot(y.T,y) - np.dot(np.dot(b.T, X.T), y)
    mse = sse/(n-p)
    
    # calculate F statistic
    F = (Q/r)/mse
    
    # critical value
    F_crit = st.f.ppf(1-alpha, r, (n-p))
    
    # p value 
    p_value = st.f.pdf(F, r, (n-p))
    
    results = {
        'F statistic': F,
        'F critical': F_crit,
        'p value': p_value
    }
    
    return results

In [237]:
L = np.array([0,1,1,1])
c = np.array([0])
r = LB_F_statistics(L, X, y, c)
r

{'F statistic': array([1.22887489]),
 'F critical': 2.594263371345763,
 'p value': array([0.40385809])}